# Measure Bin Guest Count 1 Heuristics Accuracy

Our occasions for bin 1:

1. `Lunch`
2. `Dinner`
3. `Casual Drink and Meal`
4. `Drinking`
5. `Not 1`
6. `Unknown`

We're going to pick specific labeled tables, and see how our heuristics is doing.

In [1]:
data_map = {
    "hockey": "../data/hockey_3_text_processed.csv",
    "valentine": "../data/valentine_3_text_processed.csv",
    "silvester": "../data/silvester_3_text_processed.csv"
}

In [2]:
LUNCH = "Lunch"
DINNER = "Dinner"
DRINKING = "Drinking"
CASUAL_DRINK_MEAL = "Casual Drink and Meal"
NOT_1 = "Not Category 1"
UNK = "Unknown"

The following are taken from the *annotations/occasions_annotations_bin1 notebook.

The annotations are split between dev and test.

### Dev

In [3]:
labeled_tables_dev_map = {
    "hockey": [
        (512690383, CASUAL_DRINK_MEAL),
        (521702519, DRINKING),
        (521093892, DRINKING),
        (520105316, NOT_1),
        (521783372, DINNER),
        (521769692, CASUAL_DRINK_MEAL),
        (512852707, CASUAL_DRINK_MEAL),
        (525538989, CASUAL_DRINK_MEAL)
    ],
    "valentine": [
        (434780854,DRINKING),
        (434753224,DINNER),
        (447023765,LUNCH),
        (434766693,CASUAL_DRINK_MEAL),
        (446799837,DINNER)
    ],
    "silvester": [
        (363436344,DINNER),
        (361891515,DINNER),
        (361024418,DINNER),
        (360026487,DINNER),
        (359644172,DRINKING) 
    ]
}

### Test

In [4]:
labeled_tables_test_map = {
    "hockey": [
        (520256354,DRINKING),
        (514474566,CASUAL_DRINK_MEAL),
        (522554624,NOT_1),
        (522777463,DRINKING),
        (519619252,CASUAL_DRINK_MEAL),
        (519598706,DINNER),
        (512856854,DRINKING),
        (520171895,DINNER)
    ]
}

------

In [5]:
import pandas as pd

Import the classifier function `1 Classifier`:

In [7]:
from bin_1 import Bin1Classifier as Classifier
classifier = Classifier()

-----

### Results per Table:

In [8]:
tables = ["hockey", "valentine", "silvester"]

In [9]:
results = {}
results_new = {}

In [10]:
from occasion_classifier import shrink_orders_to_table

In [11]:
for table in tables:
    print("Running for", table)
    df_path = data_map[table]
    df = pd.read_csv(df_path)
    labeled_tables = labeled_tables_dev_map[table]
    
    results[table] = []
    for order_id, true_ocassion in labeled_tables:
        orders = df[df.order_id == order_id]
        orders = shrink_orders_to_table(orders)
        pred_occasion, pred_times = classifier.classify(orders)
        results[table].append((order_id, true_ocassion, pred_occasion, ",".join(pred_times)))

Running for hockey
Running for valentine
Running for silvester


#### Show the results:

In [12]:
columns = ["order_id", "true_occasion", "pred_occasion", "pred_times"]

In [13]:
def color(data):
    correct = data["true_occasion"] in data["pred_occasion"]
    if correct:
        color = "#58f200"
    else:
        color = "#ee1300"

    return ["background-color: %s" % color] * len(data.values)

In [14]:
table = "hockey"

df = pd.DataFrame(results[table], columns=columns)
df.index += 1
df.style.apply(color, axis=1)
# df.to_csv(str(table) + "_3to5_test_results.csv", index=False)

,order_id,true_occasion,pred_occasion,pred_times
1,512690383,Casual Drink and Meal,Casual Drink and Meal,afternoon
2,521702519,Drinking,Drinking,afternoon
3,521093892,Drinking,Drinking,late_night
4,517505343,Not Category 1,Casual Drink and Meal,dinner
5,521783372,Dinner,Unknown,
6,521769692,Casual Drink and Meal,Casual Drink and Meal,dinner
7,512852707,Casual Drink and Meal,Casual Drink and Meal,late_night
8,525538989,Casual Drink and Meal,Casual Drink and Meal,afternoon


In [15]:
table = "valentine"

df = pd.DataFrame(results[table], columns=columns)
df.index += 1
df.style.apply(color, axis=1)
# df.to_csv(str(table) + "_3to5_test_results.csv", index=False)

,order_id,true_occasion,pred_occasion,pred_times
1,434780854,Drinking,Casual Drink and Meal,afternoon
2,434753224,Dinner,Unknown,
3,447023765,Lunch,Unknown,
4,434766693,Casual Drink and Meal,Casual Drink and Meal,late_night
5,446799837,Dinner,Casual Drink and Meal,dinner


In [16]:
table = "silvester"

df = pd.DataFrame(results[table], columns=columns)
df.index += 1
df.style.apply(color, axis=1)
# df.to_csv(str(table) + "_3to5_test_results.csv", index=False)

,order_id,true_occasion,pred_occasion,pred_times
1,363436344,Dinner,Unknown,
2,361891515,Dinner,Not Category 1,
3,361024418,Dinner,Casual Drink and Meal,late_night
4,360026487,Dinner,Unknown,
5,359644172,Drinking,Casual Drink and Meal,dinner


# TEST

In [24]:
tables = ["hockey"]
results = {}
results_new = {}
for table in tables:
    print("Running for", table)
    df_path = data_map[table]
    df = pd.read_csv(df_path)
    labeled_tables = labeled_tables_test_map[table]
    
    results[table] = []
    for order_id, true_ocassion in labeled_tables:
        orders = df[df.order_id == order_id]
        orders = shrink_orders_to_table(orders)
        pred_occasion, pred_times = classifier.classify(orders)
        results[table].append((order_id, true_ocassion, ",".join(pred_occasion), ",".join(pred_times)))

Running for hockey


In [25]:
table = "hockey"

df = pd.DataFrame(results[table], columns=columns)
df.index += 1
df.style.apply(color, axis=1)
# df.to_csv(str(table) + "_3to5_test_results.csv", index=False)

,order_id,true_occasion,pred_occasion,pred_times
1,520256354,Drinking,Drinking,dinner
2,514474566,Casual Drink and Meal,"Dinner,Casual Drink and Meal",afternoon
3,522554624,Not Category 1,"Not Category 1,Dinner,Casual Drink and Meal",afternoon
4,522777463,Drinking,Casual Drink and Meal,late_night
5,519619252,Casual Drink and Meal,"Drinking,Casual Drink and Meal",dinner
6,519598706,Dinner,Dinner,late_night
7,512856854,Drinking,"Dinner,Drinking,Casual Drink and Meal",
8,520171895,Dinner,Dinner,dinner
